# Practical Exercise 4.02: Bond Optimal Portfolio

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load the bond data from the Excel file

file_path = 'Set of bonds.xlsx'
df_bonds = pd.read_excel(file_path)

# Asking for bonds for the portfolio

portfolio_bonds_input = input("Enter the bond names for the portfolio, separated by commas (e.g., 'Bond A, Bond C'): ")


# Convert user input into a bond list

portfolio_bonds = [bond.strip() for bond in portfolio_bonds_input.split(',')]

# Make sure that the bonds entered exist in the data

for bond in portfolio_bonds:
    if bond not in df_bonds['Bond'].values:
        raise ValueError(f"Bond {bond} not found in the bond table.")

# Request total investment

total_investment = float(input("Enter the total investment amount (e.g., 1000000): "))

# Target modified duration

target_modified_duration = float(input("Enter the target modified duration: "))


# Selected bonds for the portfolio

selected_bonds = df_bonds[df_bonds['Bond'].isin(portfolio_bonds)]

# Modified durations and YTMs of selected bonds

modified_durations = selected_bonds['Modified Duration'].values
ytms = selected_bonds['YTM (%)'].values

# Objective function: to maximize the portfolio's YTM while targeting a specific modified duration

def objective(weights):
    portfolio_modified_duration = np.dot(weights, modified_durations)
    # Penalty for deviating from target duration
    penalty = (portfolio_modified_duration - target_modified_duration) ** 2
    portfolio_ytm = np.dot(weights, ytms)
    return -portfolio_ytm + penalty  # Negative because we are maximizing

# Constraint: the sum of the weights must be 100%

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Additional constraints: weights must be non-negative (between 0% and 100%)

bounds = [(0, 1) for _ in range(len(modified_durations))]

# Initial weights (equal to start)

initial_weights = np.full(len(modified_durations), 1 / len(modified_durations))


# Solving the optimization

results = minimize(objective, initial_weights, bounds=bounds, constraints=constraints)

# Extracting optimal weights

optimal_weights = results.x

# Calculate the investment in euros and number of securities for the portfolio

investment_portfolio = optimal_weights * total_investment

# Always round up

securities_portfolio = np.ceil(investment_portfolio / selected_bonds['Price'].values)

# Calculate final investment

final_investment_portfolio = securities_portfolio * selected_bonds['Price'].values

# Calculate YTM (IRR), convexity, cost, and modified duration of the portfolio using the optimal weights

ytm_optimal_portfolio = np.dot(optimal_weights, selected_bonds['YTM (%)'].values)
convexity_optimal_portfolio = np.dot(optimal_weights, selected_bonds['Convexity'].values)
cost_optimal_portfolio = np.dot(optimal_weights, selected_bonds['Price'].values)
modified_duration_optimal_portfolio = np.dot(optimal_weights, selected_bonds['Modified Duration'].values)

# Create a consolidated table with all portfolio information

table_results_portfolio = pd.DataFrame({
    'Bond': selected_bonds['Bond'].values,
    'Weight (%)': optimal_weights * 100,  # Convert weights to %
    'Investment €': investment_portfolio,
    'Price €': selected_bonds['Price'].values,
    'Securities': securities_portfolio,
    'Final Investment €': final_investment_portfolio,
    'YTM (%)': selected_bonds['YTM (%)'].values * 100,  # Convert YTM to %
    'Modified Duration': selected_bonds['Modified Duration'].values,
    'Convexity': selected_bonds['Convexity'].values
})

# Create a row for the portfolio

row_portfolio = pd.DataFrame({
    'Bond': ['Optimal Portfolio'],
    'Weight (%)': [100.0],  # The complete portfolio weighs 100%
    'Investment €': [investment_portfolio.sum()],
    'Price €': [np.nan],  # Leave empty to not apply formatting
    'Securities': [np.nan],  # Leave empty to not apply formatting
    'Final Investment €': [final_investment_portfolio.sum()],
    'YTM (%)': [ytm_optimal_portfolio * 100],  # Convert YTM to %
    'Modified Duration': [modified_duration_optimal_portfolio],
    'Convexity': [convexity_optimal_portfolio]
})

# Concatenate all rows of the portfolio to the result table

table_results = pd.concat([table_results_portfolio, row_portfolio], ignore_index=True)

# Format all columns to display values to two decimal places,
# excluding NaN cells

table_results['Weight (%)'] = table_results['Weight (%)'].map('{:.2f}%'.format)
table_results['Investment €'] = table_results['Investment €'].map('{:,.2f}'.format)
table_results['Price €'] = table_results['Price €'].map(lambda x: '{:,.2f}'.format(x) if pd.notnull(x) else '')
table_results['Securities'] = table_results['Securities'].map(lambda x: '{:,.0f}'.format(x) if pd.notnull(x) else '')
table_results['Final Investment €'] = table_results['Final Investment €'].map('{:,.2f}'.format)
table_results['YTM (%)'] = table_results['YTM (%)'].map('{:.2f}%'.format)
table_results['Modified Duration'] = table_results['Modified Duration'].map('{:.2f}'.format)
table_results['Convexity'] = table_results['Convexity'].map('{:.2f}'.format)

# Show the complete table

table_results
